In [1]:
import argparse
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
from intern.utils.parallel import block_compute
import configparser
import requests
import numpy as np
from numpy import genfromtxt
import shutil
import blosc
from IPython.core.debugger import set_trace
import sys
import os
import itertools
from functools import partial
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import csv
import datetime
import toolbox

In [21]:
def main(COLL_NAME, EXP_NAME, COORD_FRAME,
         CHAN_NAMES=None, num_threads = 4, CONFIG_FILE= 'config.ini'):

    bf = [5,245,245] # in z,y,x order

    config = configparser.ConfigParser()
    config.read(CONFIG_FILE)
    TOKEN = config['Default']['token']
    boss_url = ''.join( ( config['Default']['protocol'],'://',config['Default']['host'],'/v1/' ) )
    #print(boss_url)
    #'https://api.boss.neurodata.io/v1/'
    
    #intern
    rem = BossRemote(CONFIG_FILE)

    cf = CoordinateFrameResource(str(COLL_NAME + '_' + EXP_NAME))
    cfr = rem.get_project(cf)
    anno_res = ChannelResource('annotation', COLL_NAME, EXP_NAME, 'annotation', datatype='uint64')
    
    ex = {'x': cfr.x_stop, 'y': cfr.y_stop, 'z': cfr.z_stop}
    
    blocks = block_compute(0,ex['x'],0,ex['y'],0,ex['z'],
               origin = (0,0,0), block_size = (512, 512, 16))

    rid = []
    for b in blocks:
      rid = rid + rem.get_ids_in_region(anno_res, 0, b[0], b[1], b[2], [0,1])

    u = np.unique(np.asarray(rid))

    ## bounding box for annotation_i
    #bb = [rem.get_bounding_box(anno_res, 0,ui, 'tight') for ui in u]
#
    #for i in range(len(bb)):
    #    bb[i]["id"] = u[i]
#
    #A = [(rem.get_cutout(
    #      anno_res, 0, bb[i]["x_range"], 
    #      bb[i]["y_range"], bb[i]["z_range"], 
    #      id_list = [bb[i]['id']]) != 0).astype(int) 
    #    for i in range(len(bb))] 
#
    ##Bmeans = [np.int32(np.round(np.mean(np.asarray(np.where(A[i] == True)),1))) for i in range(len(A))]
    #Bmeans = [np.int32(np.round(np.mean(np.asarray(np.where(A[i] == 1)),1))) for i in range(len(A))]
    #
    #Bglobal = []
    #for i in range(len(bb)):
    #    ad1 = np.asarray([bb[i]['z_range'][0], bb[i]['y_range'][0], bb[i]['x_range'][0]])
    #    Bglobal.append(Bmeans[i] + ad1)
    #    
    #ColMin = np.asarray(bf)
    #ColMax = np.asarray([ex['z'] - (bf[0] + 1),  # The z index is inclusive
    #                     ex['y'] - (bf[1] + 1), 
    #                     ex['x'] - (bf[2] + 1)])
#
    #m = [Bglobal[i] >= ColMin for i in range(len(Bglobal))]
    #M = [Bglobal[i] <= ColMax for i in range(len(Bglobal))]
    #mm = [np.all(m[i]) for i in range(len(m)) ]
    #MM = [np.all(M[i]) for i in range(len(M)) ]
#
    #Bcon = []
    #con = [np.asarray(mm[j] and MM[j]) for j in range(len(mm))]
    #for i in range(len(Bglobal)):
    #    if con[i]: 
    #        Bcon.append(Bglobal[i])
#
    #loc = np.asarray(Bcon)
    return(u)
## END main

In [22]:
def testMain():
    COLL_NAME      = 'collman' 
    EXP_NAME       = 'collman15v2' 
    COORD_FRAME    = 'collman_collman15v2'
    CONFIG_FILE    = 'config.ini'
    OUTPUT         = 'fmaxTest20171214.csv'

    CHAN_NAMES = ['Synapsin647', 'VGluT1_647']
    #CHAN_NAMES = ['DAPI1st', 'DAPI2nd', 'DAPI3rd', 'GABA488', 'GAD647',
    #        'gephyrin594', 'GS594', 'MBP488', 'NR1594', 'PSD95_488',
    #        'Synapsin647', 'VGluT1_647']
    #CHAN_NAMES = ['synapsin', 'PSDr'] 

    Bcon  = main(COLL_NAME, EXP_NAME, COORD_FRAME,
         CHAN_NAMES=CHAN_NAMES, num_threads = 6, CONFIG_FILE= 'config.ini')

    return(Bcon)
## End testMain   

In [23]:
Bcon = testMain() 

In [24]:
len(Bcon)

236

In [25]:
Bcon

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 18